# script arguments

In [3]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:90% !important; }</style>"))

In [45]:
from statistics import median

In [ ]:
import sys

# argument list:
# - stemmed: true or false
# - representation: BOW, TFIDF, LDA_10, LDA_25, LDA_50, Word2Vec, Glove, Doc2Vec
# - technique: full, elkan
# - num_clusters: 10, 50, 100, 500, 1000

stemmed = sys.argv[1]
representation = sys.argv[2]
technique = sys.argv[3]
num_clusters = int(sys.argv[4])

if (stemmed not in ['true', 'false']):
    print('incorrect kmeans stemmed argument value: ' + stemmed)
    quit()
    
if (representation not in ['BOW', 'TFIDF', 'LDA_10', 'LDA_25', 'LDA_50', 'Word2Vec', 'Glove', 'Doc2Vec']):
    print('incorrect kmeans representation argument value: ' + representation)
    quit()
    
if (technique not in ['full', 'elkan']):
    print('incorrect kmeans technique argument value: ' + technique)
    quit()
    
if (num_clusters not in [5, 10, 20, 40]):
    print('incorrect kmeans num_clusters argument value: ' + num_clusters)
    quit()

In [51]:
stemmed = 'true'

representation = 'BOW' # 
#representation = 'TFIDF' # 
#representation = 'LDA_10' # good :)
#representation = 'LDA_25' # 
#representation = 'LDA_50' # 
#representation = 'Word2Vec' #
#representation = 'Glove' # 
#representation = 'Doc2Vec' #

technique = 'full'
num_clusters = 5

In [33]:
stemmed = 'false'

#representation = 'BOW' # 
#representation = 'TFIDF' # 
#representation = 'LDA_10' # 
#representation = 'LDA_25' # 
#representation = 'LDA_50' # 
representation = 'Word2Vec' #
#representation = 'Glove' # 
#representation = 'Doc2Vec' #

technique = 'full'
num_clusters = 40

In [2]:
from clustering_and_metrics_kmeans import bow_model, tfidf_model, lda_10_model, lda_25_model, lda_50_model, w2v_model, gv_model, d2v_model
from WebisKmeansMetricResults import KmeansMetricResults
import numpy as np
import pickle
import time

## creating and saving result object

In [3]:
def calculate_and_save_metrics(kmeans_results, time_1, time_2):
    # interdistance mean
    print('calculating interdistance mean')
    kmeans_centroid_centroid_distances, kmeans_inter_dist = kmeans_results.intercluster_dist()
    kmeans_results_inter_dist_mean = np.average(kmeans_inter_dist)
    
    # silhouette coefficient
    print('calculating silhouettte')
    a = np.mean(kmeans_results.intra_avg_list)
    b = kmeans_results_inter_dist_mean
    silhouette_score = (b - a)/np.max([a, b])
    
    # silhouette coefficient per cluster
    silhouette_cluster_map = {}
    for cluster in range(len(kmeans_results.intra_avg_list)):
        a = kmeans_results.intra_avg_list[cluster]
        b = kmeans_inter_dist[cluster]
        score = (b - a)/np.max([a, b])
        silhouette_cluster_map[cluster] = score
    
    # creating result object
    print('creating metrics object')
    kmeansMetricResults = KmeansMetricResults(stemmed, representation, technique, num_clusters)
    kmeansMetricResults.set_intra_metrics(np.average(kmeans_results.intra_avg_list), kmeans_results.intra_avg_map, np.average(kmeans_results.intra_variance_list), kmeans_results.intra_variance_map)
    kmeansMetricResults.set_inter_metrics(kmeans_inter_dist, kmeans_results_inter_dist_mean)
    kmeansMetricResults.set_silhouette(silhouette_score, silhouette_cluster_map)
    
    # calculate hitsd
    kmeans_results.calculate_total_hits()
    kmeans_results.calculate_paraphrase_hits()
    kmeans_results.calculate_plagiarize_hits()
    
    # save hits
    kmeansMetricResults.set_total_hit_metrics(kmeans_results.total_checks, kmeans_results.total_checks_map, kmeans_results.total_hits, kmeans_results.total_hits_map, kmeans_results.total_hit_percent, kmeans_results.total_hit_percent_map)
    kmeansMetricResults.set_paraphrase_hit_metrics(kmeans_results.paraphrase_checks, kmeans_results.paraphrase_checks_map, kmeans_results.paraphrase_hits, kmeans_results.paraphrase_hits_map, kmeans_results.paraphrase_hit_percent, kmeans_results.paraphrase_hit_percent_map)
    kmeansMetricResults.set_plagiarize_hit_metrics(kmeans_results.plagiarize_checks, kmeans_results.plagiarize_checks_map, kmeans_results.plagiarize_hits, kmeans_results.plagiarize_hits_map, kmeans_results.plagiarize_hit_percent, kmeans_results.plagiarize_hit_percent_map)
    
    # set computing time metrics
    time_3 = round(time.time() * 1000)
    kmeansMetricResults.set_compute_time(time_2-time_1, time_3-time_2)
    
    # saving results
    print('saving metrics object')
    kmeansMetricResults.save()

## bag of words

In [53]:
def new_bow_kmeans(): 
    bow_stemmed_kmeans = bow_model(stemmed, technique, num_clusters)
    bow_stemmed_kmeans.load_data()
    time_1 = round(time.time() * 1000)
    bow_stemmed_kmeans.run_kmeans()
    time_2 = round(time.time() * 1000)
    bow_stemmed_kmeans.calculate_intracluster_similarity()
    return bow_stemmed_kmeans, time_1, time_2

In [ ]:
if representation == 'BOW':
    kmeans_results, time_1, time_2 = new_bow_kmeans()
    calculate_and_save_metrics(kmeans_results, time_1, time_2)

In [50]:
#40
kmeans_results, time_1, time_2 = new_bow_kmeans()
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

using file extension .npz for bow/stemmed_bow_matrix
loading ../../processed_files/bow/stemmed_bow_matrix.npz
loading ../../processed_files/webis/df.pickle
finding 40 clusters with full algorithm


(11.0, 392.95)

In [84]:
stemmed = 'true'
representation = 'BOW'
technique = 'full'
num_clusters = 5
kmeans_results, time_1, time_2 = new_bow_kmeans()

using file extension .npz for bow/stemmed_bow_matrix
loading ../../processed_files/bow/stemmed_bow_matrix.npz
loading ../../processed_files/webis/df.pickle
finding 5 clusters with full algorithm


In [85]:
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

(1332, 3143.6)

In [86]:
pickle.dump(kmeans_results.cluster_list, open('kmeans_bow_true_full_5_cluster_list', 'wb'))

In [87]:
kmeans_results.cluster_list

{0: [(0, 3273),
  (0, 4026),
  (0, 4028),
  (0, 4029),
  (0, 4030),
  (0, 4031),
  (0, 4032),
  (0, 5021),
  (0, 5026),
  (0, 5088),
  (0, 5092),
  (0, 5094),
  (0, 5119),
  (0, 5214),
  (0, 5215),
  (0, 5220),
  (0, 5221),
  (0, 5222),
  (0, 5223),
  (0, 5242),
  (0, 5243),
  (0, 5244),
  (0, 5248),
  (0, 5249),
  (0, 5250),
  (0, 5274),
  (0, 5275),
  (0, 5276),
  (0, 5300),
  (0, 5302),
  (0, 5303),
  (0, 5360),
  (0, 5361),
  (0, 5397),
  (0, 5412),
  (0, 5413),
  (0, 5414),
  (0, 5415),
  (0, 5418),
  (0, 5428),
  (0, 5430),
  (0, 5434),
  (0, 5436),
  (0, 5452),
  (0, 5453),
  (0, 5457),
  (0, 5480),
  (0, 5482),
  (0, 5483),
  (0, 5484),
  (0, 5494),
  (0, 5495),
  (0, 5496),
  (0, 5497),
  (0, 5507),
  (0, 5510),
  (0, 5514),
  (0, 5515),
  (0, 5590),
  (0, 5591),
  (0, 5592),
  (0, 5593),
  (0, 5594),
  (0, 5612),
  (0, 5613),
  (0, 5614),
  (0, 5629),
  (0, 5630),
  (0, 5632),
  (0, 5633),
  (0, 5634),
  (0, 5636),
  (0, 5637),
  (0, 5640),
  (0, 5641),
  (0, 5642),
  (0, 564

## tfidf matrix

In [55]:
def new_tfidf_kmeans():
    tfidf_kmeans = tfidf_model(stemmed, technique, num_clusters)
    tfidf_kmeans.load_data()
    time_1 = round(time.time() * 1000)
    tfidf_kmeans.run_kmeans()
    time_2 = round(time.time() * 1000)
    tfidf_kmeans.calculate_intracluster_similarity()
    return tfidf_kmeans, time_1, time_2

In [ ]:
if representation == 'TFIDF':
    kmeans_results, time_1, time_2 = new_tfidf_kmeans()
    calculate_and_save_metrics(kmeans_results, time_1, time_2)

In [60]:
stemmed = 'true'
representation = 'TFIDF'
technique = 'full'
num_clusters = 5
kmeans_results, time_1, time_2 = new_tfidf_kmeans()

using file extension .npz for tfidf/tfidf_matrix
loading ../../processed_files/tfidf/tfidf_matrix.npz
loading ../../processed_files/webis/df.pickle
finding 5 clusters with full algorithm


In [61]:
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

(13, 3057.2)

In [62]:
pickle.dump(kmeans_results.cluster_list, open('kmeans_tfidf_false_full_5_cluster_list', 'wb'))

# LDA

## lda 10 matrix

In [79]:
def new_lda_10_kmeans():
    lda_10_kmeans = lda_10_model(stemmed, technique, num_clusters)
    lda_10_kmeans.load_data()
    time_1 = round(time.time() * 1000)
    lda_10_kmeans.run_kmeans()
    time_2 = round(time.time() * 1000)
    lda_10_kmeans.calculate_intracluster_similarity()
    return lda_10_kmeans, time_1, time_2

In [6]:
if representation == 'LDA_10':
    kmeans_results, time_1, time_2 = new_lda_10_kmeans()
    calculate_and_save_metrics(kmeans_results, time_1, time_2)

using file extension .npy for lda_10/stemmed_lda_10_matrix
loading ../../processed_files/lda_10/stemmed_lda_10_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 20 clusters with full algorithm
calculating interdistance mean
calculating silhouettte
creating metrics object
saving metrics object


In [96]:
stemmed = 'false'
representation = 'LDA_10'
technique = 'full'
num_clusters = 5
kmeans_results, time_1, time_2 = new_lda_10_kmeans()

using file extension .npy for lda_10/lda_10_matrix
loading ../../processed_files/lda_10/lda_10_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 5 clusters with full algorithm


In [98]:
intraDistClusterList = []

for i in range(len(kmeans_results.cluster_list)):
    print(i)
    intraDistClusterList.append((kmeans_results.intra_avg_map[i], list(kmeans_results.cluster_list[i])))
    
intraDistClusterList = sorted(intraDistClusterList)

0
1
2
3
4


In [99]:
intraDistClusterList

[(0.2770521420749306,
  [(4, 28),
   (4, 29),
   (4, 52),
   (4, 53),
   (4, 54),
   (4, 133),
   (4, 134),
   (4, 137),
   (4, 138),
   (4, 139),
   (4, 140),
   (4, 141),
   (4, 176),
   (4, 200),
   (4, 220),
   (4, 223),
   (4, 224),
   (4, 231),
   (4, 232),
   (4, 233),
   (4, 234),
   (4, 250),
   (4, 266),
   (4, 275),
   (4, 284),
   (4, 343),
   (4, 411),
   (4, 412),
   (4, 475),
   (4, 493),
   (4, 494),
   (4, 526),
   (4, 599),
   (4, 601),
   (4, 603),
   (4, 604),
   (4, 611),
   (4, 612),
   (4, 613),
   (4, 614),
   (4, 615),
   (4, 616),
   (4, 656),
   (4, 677),
   (4, 720),
   (4, 722),
   (4, 730),
   (4, 754),
   (4, 774),
   (4, 775),
   (4, 776),
   (4, 777),
   (4, 812),
   (4, 813),
   (4, 816),
   (4, 817),
   (4, 885),
   (4, 887),
   (4, 888),
   (4, 898),
   (4, 900),
   (4, 943),
   (4, 944),
   (4, 965),
   (4, 987),
   (4, 1003),
   (4, 1017),
   (4, 1027),
   (4, 1028),
   (4, 1054),
   (4, 1055),
   (4, 1115),
   (4, 1180),
   (4, 1214),
   (4, 1215)

In [100]:
pickle.dump(intraDistClusterList, open('kmeans_lda_10_cluster_5-intra_dist_cluster_list.results', 'wb'))

In [91]:
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

(1002, 3057.2)

In [92]:
pickle.dump(kmeans_results.cluster_list, open('kmeans_lda_10_false_full_5_cluster_list', 'wb'))

In [8]:
plagiarize_checks = 0
plagiarize_hits = 0
plagiarize_hit_percent = 0
plagiarize_indices = list(kmeans_results.df_merged['paraphrase'][kmeans_results.df_merged['paraphrase'] == False].index)

In [9]:
plagiarize_indices

[40,
 41,
 42,
 43,
 50,
 51,
 52,
 53,
 62,
 63,
 64,
 65,
 66,
 67,
 70,
 71,
 72,
 73,
 80,
 81,
 90,
 91,
 96,
 97,
 106,
 107,
 108,
 109,
 114,
 115,
 124,
 125,
 138,
 139,
 140,
 141,
 148,
 149,
 176,
 177,
 192,
 193,
 196,
 197,
 200,
 201,
 208,
 209,
 214,
 215,
 220,
 221,
 222,
 223,
 226,
 227,
 230,
 231,
 236,
 237,
 258,
 259,
 260,
 261,
 262,
 263,
 272,
 273,
 276,
 277,
 286,
 287,
 294,
 295,
 296,
 297,
 298,
 299,
 304,
 305,
 320,
 321,
 322,
 323,
 332,
 333,
 340,
 341,
 342,
 343,
 344,
 345,
 350,
 351,
 356,
 357,
 358,
 359,
 374,
 375,
 384,
 385,
 386,
 387,
 388,
 389,
 394,
 395,
 400,
 401,
 410,
 411,
 418,
 419,
 422,
 423,
 426,
 427,
 430,
 431,
 432,
 433,
 442,
 443,
 446,
 447,
 450,
 451,
 456,
 457,
 466,
 467,
 472,
 473,
 478,
 479,
 484,
 485,
 486,
 487,
 494,
 495,
 506,
 507,
 512,
 513,
 514,
 515,
 518,
 519,
 524,
 525,
 536,
 537,
 556,
 557,
 558,
 559,
 560,
 561,
 576,
 577,
 584,
 585,
 592,
 593,
 596,
 597,
 598,
 599,
 604

In [10]:
kmeans_results.df_merged.head()

,file_number,file_type,filename,text,paraphrase,text_processed,text_processed_stemmed
0,1,original.txt,1-original.txt,M. Comte would not advise so irrational a proc...,True,"[comte, would, advise, irrational, proceeding,...","[comt, would, advis, irrat, proceed, comt, con..."
1,1,paraphrase.txt,1-paraphrase.txt,Even M. Comte would spurn such irrational reas...,True,"[even, comte, would, spurn, irrational, reason...","[even, comt, would, spurn, irrat, reason, howe..."
2,10,original.txt,10-original.txt,Picture illustrating the shortest and easiest ...,True,"[picture, illustrating, shortest, easiest, way...","[pictur, illustr, shortest, easiest, way, catc..."
3,10,paraphrase.txt,10-paraphrase.txt,This picture exemplifies the straight and easi...,True,"[picture, exemplifies, straight, easiest, way,...","[pictur, exemplifi, straight, easiest, way, re..."
4,100,original.txt,100-original.txt,"Thad, of course. And, Bill, we're going to get...",True,"[thad, course, bill, going, get, sooner, later...","[thad, cours, bill, go, get, sooner, later, mr..."


In [12]:
#kmeans_results.cluster_list[0]

In [16]:
cluster_docs = []
for i in kmeans_results.cluster_list[0]:
    if i[1] in plagiarize_indices:
        cluster_docs.append(i[1])

In [18]:
#cluster_docs

In [19]:
for cluster in range(len(kmeans_results.cluster_list)):
    cluster_docs = [i[1] for i in kmeans_results.cluster_list[cluster] if i[1] in plagiarize_indices]
    original_docs = [i for i in cluster_docs if i % 2 == 0]
    plagiarize_docs = [i for i in cluster_docs if i % 2 == 1]

    checks = len(original_docs)
    print('checks', checks)
    kmeans_results.plagiarize_checks += checks
    kmeans_results.plagiarize_checks_map[cluster] = checks

    hits = len(set(original_docs).intersection(set(plagiarize_docs)))
    print('hits', hits)
    kmeans_results.plagiarize_hits += hits
    kmeans_results.plagiarize_hits_map[cluster] = hits

    # was total_plagiarize_hit_percent_map
    kmeans_results.plagiarize_hit_percent_map[cluster] = hits / (checks if checks > 0 else 999999)
    
    break
kmeans_results.plagiarize_hit_percent = kmeans_results.plagiarize_hits / (kmeans_results.plagiarize_checks if kmeans_results.plagiarize_checks > 0 else 999999)
print('hit percent', kmeans_results.plagiarize_hits / (kmeans_results.plagiarize_checks if kmeans_results.plagiarize_checks > 0 else 999999))

checks 85
hits 0
hit percent 0.0


In [25]:
#paraphrase_indices = list(kmeans_results.df_merged['paraphrase'][kmeans_results.df_merged['paraphrase'] == True].index)

In [26]:
#plagiarize_indices = list(kmeans_results.df_merged['paraphrase'][kmeans_results.df_merged['paraphrase'] == False].index)

In [27]:
#paraphrase_indices

In [28]:
#plagiarize_indices

## lda 25 matrix

In [75]:
def new_lda_25_kmeans():
    lda_25_kmeans = lda_25_model(stemmed, technique, num_clusters)
    lda_25_kmeans.load_data()
    time_1 = round(time.time() * 1000)
    lda_25_kmeans.run_kmeans()
    time_2 = round(time.time() * 1000)
    lda_25_kmeans.calculate_intracluster_similarity()
    return lda_25_kmeans, time_1, time_2

In [ ]:
if representation == 'LDA_25':
    kmeans_results, time_1, time_2 = new_lda_25_kmeans()
    calculate_and_save_metrics(kmeans_results, time_1, time_2)

In [76]:
stemmed = 'false'
representation = 'LDA_25'
technique = 'full'
num_clusters = 5
kmeans_results, time_1, time_2 = new_lda_25_kmeans()

using file extension .npy for lda_25/lda_25_matrix
loading ../../processed_files/lda_25/lda_25_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 5 clusters with full algorithm


In [77]:
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

(1635, 3057.2)

## lda 50 matrix

In [72]:
def new_lda_50_kmeans():
    lda_50_kmeans = lda_50_model(stemmed, technique, num_clusters)
    lda_50_kmeans.load_data()
    time_1 = round(time.time() * 1000)
    lda_50_kmeans.run_kmeans()
    time_2 = round(time.time() * 1000)
    lda_50_kmeans.calculate_intracluster_similarity()
    return lda_50_kmeans, time_1, time_2

In [ ]:
if representation == 'LDA_50':
    kmeans_results, time_1, time_2 = new_lda_50_kmeans()
    calculate_and_save_metrics(kmeans_results, time_1, time_2)

In [73]:
stemmed = 'false'
representation = 'LDA_50'
technique = 'full'
num_clusters = 5
kmeans_results, time_1, time_2 = new_lda_50_kmeans()

using file extension .npy for lda_50/lda_50_matrix
loading ../../processed_files/lda_50/lda_50_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 5 clusters with full algorithm


In [74]:
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

(1903, 3057.2)

## word2vec matrix

In [25]:
def new_w2v_kmeans():
    w2v_kmeans = w2v_model(stemmed, technique, num_clusters)
    w2v_kmeans.load_data()
    time_1 = round(time.time() * 1000)
    w2v_kmeans.run_kmeans()
    time_2 = round(time.time() * 1000)
    w2v_kmeans.calculate_intracluster_similarity()
    return w2v_kmeans, time_1, time_2

In [ ]:
if representation == 'Word2Vec':
    kmeans_results, time_1, time_2 = new_w2v_kmeans()
    calculate_and_save_metrics(kmeans_results, time_1, time_2)

In [34]:
kmeans_results, time_1, time_2 = new_w2v_kmeans()

using file extension .npy for w2v/w2v_matrix
loading ../../processed_files/w2v/w2v_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 40 clusters with full algorithm


In [63]:
stemmed = 'false'
representation = 'Word2Vec'
technique = 'full'
num_clusters = 5
kmeans_results, time_1, time_2 = new_w2v_kmeans()

using file extension .npy for w2v/w2v_matrix
loading ../../processed_files/w2v/w2v_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 5 clusters with full algorithm


In [64]:
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

(5, 3057.2)

In [44]:
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

(2.0, 382.15)

## glove matrix

In [66]:
def new_gv_kmeans():
    gv_kmeans = gv_model(stemmed, technique, num_clusters)
    gv_kmeans.load_data()
    time_1 = round(time.time() * 1000)
    gv_kmeans.run_kmeans()
    time_2 = round(time.time() * 1000)
    gv_kmeans.calculate_intracluster_similarity()
    return gv_kmeans, time_1, time_2

In [ ]:
if representation == 'Glove':
    kmeans_results, time_1, time_2 = new_gv_kmeans()
    calculate_and_save_metrics(kmeans_results, time_1, time_2)

In [67]:
stemmed = 'false'
representation = 'Glove'
technique = 'full'
num_clusters = 5
kmeans_results, time_1, time_2 = new_gv_kmeans()

using file extension .npy for glove/glove_matrix
loading ../../processed_files/glove/glove_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 5 clusters with full algorithm


In [68]:
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

(7, 3143.6)

## doc2vec matrix

In [11]:
def new_d2v_kmeans():
    d2v_kmeans = d2v_model(stemmed, technique, num_clusters)
    d2v_kmeans.load_data()
    time_1 = round(time.time() * 1000)
    d2v_kmeans.run_kmeans()
    time_2 = round(time.time() * 1000)
    d2v_kmeans.calculate_intracluster_similarity()
    return d2v_kmeans, time_1, time_2

In [12]:
if representation == 'Doc2Vec':
    kmeans_results, time_1, time_2 = new_d2v_kmeans()
    calculate_and_save_metrics(kmeans_results, time_1, time_2)

using file extension .npy for d2v/stemmed_d2v_matrix
loading ../../processed_files/d2v/stemmed_d2v_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 40 clusters with full algorithm
calculating interdistance mean
calculating silhouettte
creating metrics object
saving metrics object


In [27]:
kmeans_results, time_1, time_2 = new_d2v_kmeans()

using file extension .npy for d2v/stemmed_d2v_matrix
loading ../../processed_files/d2v/stemmed_d2v_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 5 clusters with full algorithm


In [69]:
stemmed = 'false'
representation = 'Doc2Vec'
technique = 'full'
num_clusters = 5
kmeans_results, time_1, time_2 = new_d2v_kmeans()

using file extension .npy for d2v/d2v_matrix
loading ../../processed_files/d2v/d2v_matrix.npy
loading ../../processed_files/webis/df.pickle
finding 5 clusters with full algorithm


In [70]:
kmeans_cluster_lengths = [len(kmeans_results.cluster_list[cluster]) for cluster in kmeans_results.cluster_list]
median(kmeans_cluster_lengths), np.average(kmeans_cluster_lengths)

(1, 3057.2)

## save kmeans cluster assignments

In [14]:
import pickle
pickle.dump(kmeans_results.cluster_list, open('kmeans_40_clusters', 'wb'))

In [18]:
pickle.dump(kmeans_results.cluster_list, open('kmeans_20_clusters', 'wb'))

In [22]:
pickle.dump(kmeans_results.cluster_list, open('kmeans_10_clusters', 'wb'))

In [28]:
pickle.dump(kmeans_results.cluster_list, open('kmeans_5_clusters', 'wb'))

In [29]:
kmeans_results.cluster_list

{0: [(0, 0),
  (0, 1),
  (0, 2),
  (0, 3),
  (0, 4),
  (0, 5),
  (0, 6),
  (0, 7),
  (0, 8),
  (0, 9),
  (0, 10),
  (0, 11),
  (0, 12),
  (0, 13),
  (0, 14),
  (0, 15),
  (0, 16),
  (0, 17),
  (0, 18),
  (0, 19),
  (0, 20),
  (0, 21),
  (0, 22),
  (0, 23),
  (0, 24),
  (0, 25),
  (0, 26),
  (0, 27),
  (0, 28),
  (0, 29),
  (0, 30),
  (0, 31),
  (0, 32),
  (0, 33),
  (0, 34),
  (0, 35),
  (0, 36),
  (0, 37),
  (0, 38),
  (0, 39),
  (0, 40),
  (0, 41),
  (0, 42),
  (0, 43),
  (0, 44),
  (0, 45),
  (0, 46),
  (0, 47),
  (0, 48),
  (0, 49),
  (0, 50),
  (0, 51),
  (0, 52),
  (0, 53),
  (0, 54),
  (0, 55),
  (0, 56),
  (0, 57),
  (0, 58),
  (0, 59),
  (0, 60),
  (0, 61),
  (0, 62),
  (0, 63),
  (0, 64),
  (0, 65),
  (0, 66),
  (0, 67),
  (0, 68),
  (0, 69),
  (0, 70),
  (0, 71),
  (0, 72),
  (0, 73),
  (0, 74),
  (0, 75),
  (0, 76),
  (0, 77),
  (0, 78),
  (0, 79),
  (0, 80),
  (0, 81),
  (0, 82),
  (0, 83),
  (0, 84),
  (0, 85),
  (0, 86),
  (0, 87),
  (0, 88),
  (0, 89),
  (0, 90),
  (0, 